In [7]:
from opensoundscape.preprocess.preprocessors import BasePreprocessor, AudioToSpectrogramPreprocessor, CnnPreprocessor
import torch
import pandas as pd
from pathlib import Path
import numpy as np
import pandas as pd
from opensoundscape.torch.models import cnn
from opensoundscape.torch.architectures import cnn_architectures

In [8]:
df = pd.read_csv('./XenoCanto_Data/xeno_canto.csv', sep=';')
df['IN FILE'] = [f.split('/')[-1] for f in df['FILENAME']]
df

,FILENAME,CLIP LENGTH,Start (s),End (s),Scientific name,Common name,Confidence,IN FILE
0,/media/jacob/Audio/Training_Xeno_Canto_2022/He...,29.753479,0.0,3.0,Picus awokera,Japanese Woodpecker,0.317242,Hemitriccus-griseipectus-108055.wav
1,/media/jacob/Audio/Training_Xeno_Canto_2022/He...,29.753479,3.0,6.0,Picus awokera,Japanese Woodpecker,0.505273,Hemitriccus-griseipectus-108055.wav
2,/media/jacob/Audio/Training_Xeno_Canto_2022/He...,29.753479,6.0,9.0,Picus awokera,Japanese Woodpecker,0.312248,Hemitriccus-griseipectus-108055.wav
3,/media/jacob/Audio/Training_Xeno_Canto_2022/He...,29.753479,9.0,12.0,Picus awokera,Japanese Woodpecker,0.414672,Hemitriccus-griseipectus-108055.wav
4,/media/jacob/Audio/Training_Xeno_Canto_2022/He...,29.753479,12.0,15.0,Picus awokera,Japanese Woodpecker,0.574978,Hemitriccus-griseipectus-108055.wav
...,...,...,...,...,...,...,...,...
10605,/media/jacob/Audio/Training_Xeno_Canto_2022/Vi...,112.509396,96.0,99.0,Vireo chivi,Chivi Vireo,0.116824,Vireo-olivaceus-98648.wav
10606,/media/jacob/Audio/Training_Xeno_Canto_2022/Vi...,112.509396,99.0,102.0,Vireo olivaceus,Red-eyed Vireo,0.568682,Vireo-olivaceus-98648.wav
10607,/media/jacob/Audio/Training_Xeno_Canto_2022/Vi...,112.509396,102.0,105.0,Vireo olivaceus,Red-eyed Vireo,0.301558,Vireo-olivaceus-98648.wav
10608,/media/jacob/Audio/Training_Xeno_Canto_2022/Vi...,112.509396,105.0,108.0,Vireo chivi,Chivi Vireo,0.269658,Vireo-olivaceus-98648.wav


In [9]:
ef = pd.read_csv('./XenoCanto_Data/for_data_science_newline_fixed.csv')
gtClips = np.array(ef['IN FILE'].unique())
ef.shape

(31365, 12)

In [10]:
j = 0
ff = pd.DataFrame(columns=df.columns)
for i, row in df.iterrows():
    if(row['IN FILE'] in gtClips):
        # print(row['IN FILE'])
        ff.loc[j] = row
        df.drop(i, inplace=True)
        j += 1
print(df.shape)
df = df[df['Confidence'] >= 0.8]

(10089, 8)


In [11]:
ff

,FILENAME,CLIP LENGTH,Start (s),End (s),Scientific name,Common name,Confidence,IN FILE
0,/media/jacob/Audio/Training_Xeno_Canto_2022/He...,33.515104,3.0,6.0,Hemitriccus griseipectus,White-bellied Tody-Tyrant,0.958464,Hemitriccus-griseipectus-11801.wav
1,/media/jacob/Audio/Training_Xeno_Canto_2022/He...,33.515104,6.0,9.0,Hemitriccus griseipectus,White-bellied Tody-Tyrant,0.97362,Hemitriccus-griseipectus-11801.wav
2,/media/jacob/Audio/Training_Xeno_Canto_2022/He...,33.515104,9.0,12.0,Hemitriccus griseipectus,White-bellied Tody-Tyrant,0.965499,Hemitriccus-griseipectus-11801.wav
3,/media/jacob/Audio/Training_Xeno_Canto_2022/He...,33.515104,12.0,15.0,Hemitriccus griseipectus,White-bellied Tody-Tyrant,0.946824,Hemitriccus-griseipectus-11801.wav
4,/media/jacob/Audio/Training_Xeno_Canto_2022/He...,33.515104,15.0,18.0,Hemitriccus griseipectus,White-bellied Tody-Tyrant,0.992666,Hemitriccus-griseipectus-11801.wav
...,...,...,...,...,...,...,...,...
516,/media/jacob/Audio/Training_Xeno_Canto_2022/Vi...,39.72,36.0,39.0,Vireo olivaceus,Red-eyed Vireo,0.596736,Vireo-olivaceus-381598.wav
517,/media/jacob/Audio/Training_Xeno_Canto_2022/Vi...,11.389396,0.0,3.0,Vireo chivi,Chivi Vireo,0.938404,Vireo-olivaceus-8545.wav
518,/media/jacob/Audio/Training_Xeno_Canto_2022/Vi...,11.389396,3.0,6.0,Pycnonotus jocosus,Red-whiskered Bulbul,0.182475,Vireo-olivaceus-8545.wav
519,/media/jacob/Audio/Training_Xeno_Canto_2022/Vi...,11.389396,6.0,9.0,Vireo chivi,Chivi Vireo,0.662427,Vireo-olivaceus-8545.wav


In [12]:
len(df['FILENAME'].unique())

473

In [13]:
df['IN FILE'] = ['./XenoCanto_Data/Training_Xeno_Canto_2022/' + f for f in df['IN FILE']]
ff['IN FILE'] = ['./XenoCanto_Data/Training_Xeno_Canto_2022/' + f for f in ff['IN FILE']]

/var/folders/km/7_v861mx0cb9fp42nvllkmt00000gn/T/ipykernel_35735/3848840096.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['IN FILE'] = ['./XenoCanto_Data/Training_Xeno_Canto_2022/' + f for f in df['IN FILE']]


In [15]:
ef = ff.groupby('IN FILE', group_keys=False).apply(lambda ff: ff.sample(1))
ef.columns

Index(['FILENAME', 'CLIP LENGTH', 'Start (s)', 'End (s)', 'Scientific name',
       'Common name', 'Confidence', 'IN FILE'],
      dtype='object')

In [16]:
from opensoundscape.audio import Audio
clip_duration = 3
clip_overlap = 0
final_clip = None
clip_dir = './temp_clips'
# classes = labels
min_label_overlap = 0.1

ef_split_save = pd.DataFrame(columns=['file', 'start_time', 'end_time', 'SAMPLING RATE', 'MANUAL ID'])

cnt = 0

for i, row in ef.iterrows():
    try:
        audio = Audio.from_file(row['IN FILE'])
    except Exception:
        print(row['IN FILE'] + ' not found')
        continue
    file_name = row['IN FILE'].split('.')
    # print(file_name)
    clip_df = audio.split_and_save(
        clip_dir,
        prefix='..' + file_name[1],
        clip_duration=clip_duration,
        clip_overlap=clip_overlap,
        final_clip='remainder',
        dry_run=False
    )
    clip_df['SAMPLING RATE'] = [44100] * clip_df.shape[0]
    clip_df['MANUAL ID'] = [row['Common name']] * clip_df.shape[0]
    clip_df.reset_index(inplace=True)
    # print(clip_df.shape)
    ef_split_save = ef_split_save.append(clip_df, ignore_index=True)
    # print(all_labels.shape)
    # print('*******')
print(ef_split_save)
ef_split_save.to_csv('xeno_canto_test_data.csv')

/var/folders/km/7_v861mx0cb9fp42nvllkmt00000gn/T/ipykernel_35735/3110984884.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ef_split_save = ef_split_save.append(clip_df, ignore_index=True)
/var/folders/km/7_v861mx0cb9fp42nvllkmt00000gn/T/ipykernel_35735/3110984884.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ef_split_save = ef_split_save.append(clip_df, ignore_index=True)
/var/folders/km/7_v861mx0cb9fp42nvllkmt00000gn/T/ipykernel_35735/3110984884.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ef_split_save = ef_split_save.append(clip_df, ignore_index=True)
/var/folders/km/7_v861mx0cb9fp42nvllkmt00000gn/T/ipykernel_35735/3110984884.py:33: FutureWarning: The frame.append method is deprecated and will be rem

                                                  file start_time   end_time  \
0    ./temp_clips/../XenoCanto_Data/Training_Xeno_C...        0.0        3.0   
1    ./temp_clips/../XenoCanto_Data/Training_Xeno_C...        3.0        6.0   
2    ./temp_clips/../XenoCanto_Data/Training_Xeno_C...        6.0        9.0   
3    ./temp_clips/../XenoCanto_Data/Training_Xeno_C...        9.0       12.0   
4    ./temp_clips/../XenoCanto_Data/Training_Xeno_C...       12.0       15.0   
..                                                 ...        ...        ...   
616  ./temp_clips/../XenoCanto_Data/Training_Xeno_C...       39.0      39.72   
617  ./temp_clips/../XenoCanto_Data/Training_Xeno_C...        0.0        3.0   
618  ./temp_clips/../XenoCanto_Data/Training_Xeno_C...        3.0        6.0   
619  ./temp_clips/../XenoCanto_Data/Training_Xeno_C...        6.0        9.0   
620  ./temp_clips/../XenoCanto_Data/Training_Xeno_C...        9.0  11.389388   

    SAMPLING RATE                  MANU

/var/folders/km/7_v861mx0cb9fp42nvllkmt00000gn/T/ipykernel_35735/3110984884.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ef_split_save = ef_split_save.append(clip_df, ignore_index=True)
/var/folders/km/7_v861mx0cb9fp42nvllkmt00000gn/T/ipykernel_35735/3110984884.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ef_split_save = ef_split_save.append(clip_df, ignore_index=True)
/var/folders/km/7_v861mx0cb9fp42nvllkmt00000gn/T/ipykernel_35735/3110984884.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ef_split_save = ef_split_save.append(clip_df, ignore_index=True)


In [12]:
df = pd.read_csv('xeno_canto_test.csv')
df['MANUAL ID'].unique()

array(['White-bellied Tody-Tyrant', 'White-throated Spadebill',
       'Philippine Woodpecker', 'Water Rail', 'Blue-backed Manakin',
       "Ihering's Antwren", 'Striated Antbird', 'Bamboo Antshrike',
       'Great-billed Hermit', 'Long-billed Hermit', 'Brown Violetear',
       'White-throated Toucan', 'Yellow-throated Toucan',
       'Mountain Scops-Owl', 'Channel-billed Toucan', 'Eastern Kingbird',
       'Brown-headed Cowbird', 'Creamy-bellied Thrush',
       'Scissor-tailed Flycatcher', 'Red-eyed Vireo',
       'Yellow-bellied Flycatcher', 'Wood Thrush', 'Chivi Vireo',
       'Noronha Vireo', 'Yellow-green Vireo', 'Brown Bullfinch',
       'Black-and-white Warbler', 'Philadelphia Vireo',
       'White-browed Tapaculo'], dtype=object)